# Tarefa 3: usar o Amazon Bedrock para responder perguntas

Neste caderno, você aprenderá a usar o modelo Titan do Bedrock para fornecer respostas informativas a consultas enviando a solicitação com todo o contexto relevante ao modelo e esperando a resposta, enfrentando o desafio de fazer com que o modelo retorne respostas factuais para perguntas sem precisar preparar e indexar documentos previamente.

Este caderno simula o que a **Geração aumentada de recuperação (RAG)** faria, mas sem usar RAG de fato. Essa abordagem funciona para documentos curtos ou aplicações singleton. Sua escala pode não ser adequada para responder perguntas em nível empresarial, visto que nem todos os documentos grandes necessários cabem no prompt enviado para o modelo.

**Responder perguntas (QA)** é uma tarefa importante que envolve extrair respostas para consultas factuais feitas em linguagem natural. Normalmente, um sistema de QA processa uma consulta em uma base de conhecimento que contém dados estruturados ou não estruturados e gera uma resposta com informações precisas. Garantir um alto nível de acurácia é essencial para desenvolver um sistema de resposta a perguntas útil e confiável, principalmente para casos de uso empresariais.


## Cenário

Você tenta modelar uma situação na UmaEmpresa em que pede para que um modelo de respostas a perguntas dê informações sobre a troca de pneus para um modelo de veículo específico que ela fabrica. Primeiro, você consulta o modelo usando uma abordagem "zero-shot" para ver se ele pode dar respostas relevantes com base apenas os dados de treinamento dele.

No entanto, você percebe que o modelo parece estar "alucinando" respostas mais genéricas, conforme evidenciado quando você testa um modelo de veículo falso e obtém respostas semelhantes. Isso sugere a necessidade de aumentar o treinamento do modelo com os manuais de veículos reais da Example Company para dar detalhes sobre os pneus de cada modelo.

Neste laboratório, você simula essa abordagem de "geração aumentada de recuperação" (RAG) sem dados externos. Você fornece um trecho detalhado do manual explicando como trocar os pneus no veículo Modelo Z da UmaEmpresa. Você testa se o modelo agora pode fornecer uma resposta personalizada e precisa usando esse exemplo de conteúdo contextualizado.

## Tarefa 3.1: Configurar o ambiente

Nesta tarefa, você configurará o ambiente.

In [1]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## Tarefa 3.2: Perguntas e respostas com o conhecimento do modelo
Nesta seção, tentamos usar um modelo oferecido pelo serviço Bedrock para responder a perguntas com base no conhecimento que ele adquiriu durante a fase de treinamento.

Nesta tarefa, você usará o método invoke_model () do cliente do Amazon Bedrock. Os parâmetros obrigatórios para usar esse método são: "modelId", que representa o ARN do modelo do Amazon Bedrock; além de "body", que é o prompt de sua tarefa.

O formato do texto do prompt muda conforme o provedor do modelo de base selecionado. Você verá isso em detalhes abaixo.

```json
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

Tente usar modelos fornecidos pelo serviço Bedrock para responder perguntas com base no conhecimento adquirido durante a fase de treinamento.

In [ ]:
prompt_data = """You are an helpful assistant. Answer questions in a concise way. If you are unsure about the
answer say 'I am unsure'

Question: How can I fix a flat tire on my AnyCompany AC8?
Answer:"""

parameters = {
    "maxTokenCount":512,
    "stopSequences":[],
    "temperature":0,
    "topP":0.9
    }



## Tarefa 3.3: invocar o modelo passando o corpo JSON para gerar a resposta

In [8]:
#model configuration
body = json.dumps({
    "messages": [{"role": "user", "content": [{"text": prompt_data}]}],
    "inferenceConfig": {
        "max_new_tokens": 512,
        "temperature": 0,
        "top_p": 0.9
    }
})

modelId = "amazon.nova-lite-v1:0"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

try:
    response = bedrock_client.invoke_model(
        body=body, modelId=modelId, accept="application/json", contentType="application/json"
    )
    response_body = json.loads(response.get("body").read())
    answer = response_body["output"]["message"]["content"][0]["text"]
    print(answer.strip())
    
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

To fix a flat tire on your AnyCompany AC8, follow these steps:

1. Ensure the bike is stable and on a flat surface.
2. Remove the wheel with the flat tire.
3. Take out the inner tube.
4. Inspect the tire and inner tube for punctures or damage.
5. Patch or replace the inner tube if necessary.
6. Reinsert the inner tube into the tire.
7. Reassemble the wheel and reattach it to the bike.
8. Inflate the tire to the recommended pressure.

If you're unsure about any step, consult the user manual or seek professional help.



O modelo fornece uma resposta descrevendo o processo de troca do pneu furado do carro, mas a mesma explicação pode ser válida para qualquer carro. Infelizmente, essa não é a resposta correta para o modelo AC8 da UmaEmpresa, que não tem estepe. Isso ocorre porque o modelo foi treinado com dados que contêm instruções sobre a troca de pneus em carros.

Outro exemplo desse problema pode ser encontrado ao tentar fazer a mesma pergunta usando uma marca e modelo de carro totalmente falsos, como Amazon Tirana.

In [12]:
prompt_data = "How can I fix a flat tire on my Amazon Tirana?"
#model configuration
body = json.dumps({
    "messages": [{"role": "user", "content": [{"text": prompt_data}]}],
    "inferenceConfig": {
        "max_new_tokens": 512,
        "temperature": 0,
        "top_p": 0.9
    }
})

modelId = "amazon.nova-lite-v1:0"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

try:
    response = bedrock_client.invoke_model(
        body=body, modelId=modelId, accept="application/json", contentType="application/json"
    )
    response_body = json.loads(response.get("body").read())
    answer = response_body["output"]["message"]["content"][0]["text"]
    print(answer.strip())
    
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

Fixing a flat tire on your Amazon Tirana (assuming it's a bicycle or a similar small vehicle) involves a few steps. Here's a general guide to help you through the process:

### Tools and Materials Needed:
1. **Tire levers** (if needed)
2. **Patch kit or tire repair plug kit** (if you plan to patch the tire)
3. **New inner tube** (if the old one is damaged)
4. **Pump** (to inflate the new inner tube)
5. **A clean cloth or rag**
6. **A small container of water** (optional, for testing the patch)

### Steps to Fix a Flat Tire:

#### 1. **Remove the Wheel:**
   - If the bike has quick-release levers, open them to release the wheel.
   - If it has bolts, use a wrench to loosen the nuts and remove the wheel.

#### 2. **Remove the Tire from the Rim:**
   - Use tire levers to pry the tire off the rim. Start at the valve stem and work your way around. Be careful not to damage the inner tube.
   - If the tire is stubborn, you can use a bit of water to help lubricate the tire and make it easier t

Considerando a pergunta do prompt, o modelo é incapaz de fornecer uma resposta realista.

Para resolver esse problema e fazer com que o modelo dê respostas com base nas instruções específicas válidas para o modelo do seu carro, você pode aumentar imediatamente o conhecimento do modelo fornecendo uma base de conhecimento adicional como parte do prompt.

Vamos ver como você pode usar isso para melhorar sua aplicação.

Suponha que o trecho a seguir seja do manual do modelo AC8 da UmaEmpresa (na realidade, não é o manual real, mas trate-o como se fosse). Esse documento também é convenientemente curto o bastante para caber inteiramente na janela de contexto do Titan Large.

```plain
Tires and Tire Pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to Find Recommended Tire Pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the Tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for Using the Tire Mobility Kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact Rivesla or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible.
```

In [13]:
context = """Tires and tire pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to find recommended tire pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for using the tire mobility kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact AnyCompany or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible."""

##### Agora, transmita o trecho inteiro para o modelo com a pergunta.

In [14]:
question = "How can I fix a flat tire on my AnyCompany AC8?"
prompt_data = f"""Answer the question based only on the information provided between ## and give step by step guide.
#
{context}
#

Question: {question}
Answer:"""

### Tarefa 3.4: Invocar o modelo via boto3 para gerar a resposta

In [16]:
body = json.dumps({
    "messages": [{"role": "user", "content": [{"text": prompt_data}]}],
    "inferenceConfig": {
        "max_new_tokens": 512,
        "temperature": 0,
        "top_p": 0.9
    }
})
modelId = "amazon.nova-lite-v1:0"

try:
    response = bedrock_client.invoke_model(
        body=body, modelId=modelId, accept="application/json", contentType="application/json"
    )
    response_body = json.loads(response.get("body").read())
    answer = response_body["output"]["message"]["content"][0]["text"]
    print(answer.strip())
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

To fix a flat tire on your AnyCompany AC8, follow these steps based on the information provided:

1. **Locate the Tire Mobility Kit:**
   - Open the tailgate or trunk of your vehicle.
   - Lift up the lining of the luggage area to access the tire mobility kit.

2. **Seal the Puncture:**
   - Follow the instructions provided with the tire mobility kit to seal the puncture in the tire. This typically involves inserting a sealant into the tire to temporarily fix the leak.

3. **Reinflate the Tire:**
   - After sealing the puncture, you can reinflate the tire using an air pump. Make sure to inflate the tire to the recommended pressure, which you can find on the inflation label on the driver's side B-pillar or in your vehicle's manual.

4. **Calibrate the Tire Pressure:**
   - In the center display of your vehicle, open the "Car status" app.
   - Navigate to the "Tire pressure" tab.
   - Press the "Calibrate pressure" option and confirm the action.
   - Drive the car for a few minutes at a 

Como o modelo demora um pouco para compreender o contexto e gerar uma resposta relevante para você, isso pode resultar em uma experiência do usuário ruim, pois é necessário aguardar a resposta por alguns segundos.

O Bedrock também é compatível com o recurso de transmissão em que o serviço gera saída conforme o modelo gera tokens. Aqui, é possível ver um exemplo de como você pode implementar isso.

In [17]:
from IPython.display import display_markdown,Markdown,clear_output

In [25]:
# response with stream
body = json.dumps({
    "messages": [{"role": "user", "content": [{"text": prompt_data}]}],
    "inferenceConfig": {
        "max_new_tokens": 512,
        "temperature": 0,
        "top_p": 0.9
    }
})
modelId = "amazon.nova-lite-v1:0"

try:
    response = bedrock_client.invoke_model(
        body=body, modelId=modelId, accept="application/json", contentType="application/json"
    )
    response_body = json.loads(response.get("body").read())
    # print(json.dumps(response_body, indent=2))  # Ver estrutura
    answer = response_body["output"]["message"]["content"][0]["text"]
    print(answer.strip())
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error


To fix a flat tire on your AnyCompany AC8, follow these steps based on the information provided:

1. **Locate the Tire Mobility Kit:**
   - Open the tailgate or trunk of your vehicle.
   - Lift up the lining of the luggage area to access the tire mobility kit.

2. **Seal the Puncture:**
   - Follow the instructions provided with the tire mobility kit to seal the puncture in the tire. This typically involves inserting a sealant into the tire to temporarily fix the leak.

3. **Reinflate the Tire:**
   - After sealing the puncture, you can reinflate the tire using an air pump. Follow the instructions on the air pump to inflate the tire to the recommended pressure. Refer to the inflation label on the driver's side B-pillar or your vehicle's manual for the correct tire pressure.

4. **Calibrate the Tire Pressure:**
   - In the center display of your vehicle, open the "Car status" app.
   - Navigate to the "Tire pressure" tab.
   - Press the "Calibrate pressure" option and confirm the action

A resposta fornece instruções passo a passo resumidas sobre como trocar os pneus. 

Você aprendeu a usar a geração aumentada de recuperação (RAG) ou o processo de aumento para gerar uma resposta com curadoria adaptada ao contexto específico e às informações fornecidas.

### Experimente você mesmo
- Altere os prompts para seu caso de uso específico e avalie o resultado de diferentes modelos.
- Teste o comprimento do token para entender a latência e a responsividade do serviço.
- Aplique diferentes princípios de engenharia de prompts para gerar resultados melhores.

### Limpeza

Você concluiu este caderno. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de caderno e continue com a **Tarefa 4**.